In [1]:
library(rstatix)

# install install.packages("effectsize") if you don't have it
if (!requireNamespace("effectsize", quietly = TRUE)) {
  install.packages("effectsize")
}

# Load the effectsize package
library(effectsize)


Attaching package: 'rstatix'


The following object is masked from 'package:stats':

    filter



Attaching package: 'effectsize'


The following objects are masked from 'package:rstatix':

    cohens_d, eta_squared




In [2]:
data_dir <- file.path(getwd(), "..", "figures", "analysis_1", Sys.Date())

if (!dir.exists(data_dir)) {
  # raise error if the directory does not exist
  stmt < -paste0("The directory ", data_dir, " does not exist. Please run the analysis first.")
  print(stmt)
  stop(stmt)
}
 
# load the data
exons_df <- read.csv(file.path(data_dir, "age_of_onset_vs_exon.csv"))
# make exon a factor
exons_df$exon <- as.factor(exons_df$exon)
head(exons_df)

,individual_id,age_of_onset,exon
,<int>,<dbl>,<fct>
1,4,2,99
2,5,10,99
3,6,2,99
4,7,10,99
5,19,12,47
6,35,11,8


In [3]:
# calculate effect size kruskal wallis
kruskal_eff <- kruskal_effsize(data = exons_df, formula = age_of_onset ~ exon, ci = TRUE)
kruskal_eff

,.y.,n,effsize,conf.low,conf.high,method,magnitude
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<ord>
Kruskal-Wallis chi-squared,age_of_onset,324,0.1638973,0.13,0.28,eta2[H],large


In [4]:
domain_df <- read.csv(file.path(data_dir, "age_of_onset_vs_domain.csv"))
# make domain a factor
domain_df$domain <- as.factor(domain_df$domain)
head(domain_df)

,individual_id,age_of_onset,domain
,<int>,<dbl>,<fct>
1,4,2,TM
2,5,10,TM
3,6,2,TM
4,7,10,TM
5,19,12,BSol
6,28,6,TM


In [5]:
domain_kruskal_eff <- kruskal_effsize(data = domain_df, formula = age_of_onset ~ domain, ci = TRUE)
domain_kruskal_eff

,.y.,n,effsize,conf.low,conf.high,method,magnitude
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<ord>
Kruskal-Wallis chi-squared,age_of_onset,404,0.02093855,0.0042,0.08,eta2[H],small


In [6]:
subdomain_df <- read.csv(file.path(data_dir, "age_of_onset_vs_subdomain.csv"))
# make subdomain a factor
subdomain_df$subdomain <- as.factor(subdomain_df$subdomain)
head(subdomain_df)

,individual_id,age_of_onset,subdomain
,<int>,<dbl>,<fct>
1,4,2,pVSD
2,5,10,pVSD
3,6,2,pVSD
4,7,10,pVSD
5,19,12,BSol1
6,28,6,pVSD


In [7]:
subdomain_kruskal_eff <- kruskal_effsize(data = subdomain_df, formula = age_of_onset ~ subdomain, ci = TRUE)
subdomain_kruskal_eff

,.y.,n,effsize,conf.low,conf.high,method,magnitude
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<ord>
Kruskal-Wallis chi-squared,age_of_onset,393,0.04444228,0.02,0.12,eta2[H],small


In [8]:
exons_df[exons_df$exon %in% c(45, 34),]

,individual_id,age_of_onset,exon
,<int>,<dbl>,<fct>
23,81,7,45
182,610,5,45
183,611,5,45
269,1122,3,45
270,1123,3,45
278,1166,4,45


In [17]:
rank_biserial(age_of_onset ~ age_of_onset, data = exons_df[exons_df$exon %in% c(45, 34),])

r_rank_biserial,CI,CI_low,CI_high
<dbl>,<dbl>,<dbl>,<dbl>
1,0.95,1,1


In [10]:
d_exon <- dunn_test(age_of_onset ~ exon, data = exons_df, p.adjust.method = "bonferroni")
head(d_exon)

.y.,group1,group2,n1,n2,statistic,p,p.adj,p.adj.signif
<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>
age_of_onset,8,14,34,47,0.9161197,0.359604137,1.0000000,ns
age_of_onset,8,15,34,6,-0.3594223,0.719279195,1.0000000,ns
age_of_onset,8,37,34,9,-1.0690420,0.285050760,1.0000000,ns
age_of_onset,8,43,34,8,0.7746816,0.438527800,1.0000000,ns
age_of_onset,8,44,34,13,-0.3995358,0.689498438,1.0000000,ns
age_of_onset,8,45,34,6,-3.4779289,0.000505304,0.1167252,ns


In [43]:
# dunn_with_rank_biserial <- function(
#     df, independent_var
# ) {
#     # for each combination of exon, calculate the effect size
#     iter_comb <- combn(levels(df[[independent_var]]), 2, simplify = FALSE)

#     formula_str <- as.formula(paste("age_of_onset ~", independent_var))
#     dunn_res <- dunn_test(formula_str, data = df, p.adjust.method = "bonferroni")

#     # convert the dunn test results to a data frame
#     dunn_res_df <- as.data.frame(dunn_res)
#     # add columns, r_rank_biserial, CI, CI_low, CI_high
#     dunn_res_df$r_rank_biserial <- NA
#     dunn_res_df$CI <- NA
#     dunn_res_df$CI_low <- NA
#     dunn_res_df$CI_high <- NA

#     for (i in seq_along(iter_comb)) {
#         cmp <- iter_comb[[i]]
#         group1 <- cmp[1]
#         group2 <- cmp[2]

#         # filter the data
#         x <- df[df[[independent_var]] == group1, "age_of_onset", drop = FALSE]
#         y <- df[df[[independent_var]] == group2, "age_of_onset", drop = FALSE]

#         # calculate the effect size
#         eff_size <- rank_biserial(x = x$age_of_onset, y = y$age_of_onset)

#         # Store the effect size with the dunn test results
#         rows_to_update <- d_exon_df$group1 == exon1 & d_exon_df$group2 == exon2 | 
#                         d_exon_df$group1 == exon2 & d_exon_df$group2 == exon1

#         dunn_res_df[rows_to_update, "r_rank_biserial"] <- eff_size$r_rank_biserial
#         dunn_res_df[rows_to_update, "CI"] <- eff_size$CI
#         dunn_res_df[rows_to_update, "CI_low"] <- eff_size$CI_low
#         dunn_res_df[rows_to_update, "CI_high"] <- eff_size$CI_high
#     }

#     return(dunn_res_df)

# }


# for each combination of exon, calculate the effect size
exon_factors <- levels(exons_df$exon)
c2_exons <- combn(exon_factors, 2, simplify = FALSE)
d_exon <- dunn_test(age_of_onset ~ exon, data = exons_df, p.adjust.method = "bonferroni")

# convert the dunn test results to a data frame
d_exon_df <- as.data.frame(d_exon)
# add columns, r_rank_biserial, CI, CI_low, CI_high
d_exon_df$r_rank_biserial <- NA
d_exon_df$CI <- NA
d_exon_df$CI_low <- NA
d_exon_df$CI_high <- NA


for (i in seq_along(c2_exons)) {
  exon_cmp <- c2_exons[[i]]
  exon1 <- exon_cmp[1]
  exon2 <- exon_cmp[2]

  # filter the data
  x <- exons_df[exons_df$exon == exon1, "age_of_onset", drop = FALSE]
  y <- exons_df[exons_df$exon == exon2, "age_of_onset", drop = FALSE]

  # calculate the effect size
  eff_size <- rank_biserial(x = x$age_of_onset, y = y$age_of_onset)

  # Store the effect size with the dunn test results
  rows_to_update <- d_exon_df$group1 == exon1 & d_exon_df$group2 == exon2 |
    d_exon_df$group1 == exon2 & d_exon_df$group2 == exon1

  d_exon_df[rows_to_update, "r_rank_biserial"] <- eff_size$r_rank_biserial
  d_exon_df[rows_to_update, "CI"] <- eff_size$CI
  d_exon_df[rows_to_update, "CI_low"] <- eff_size$CI_low
  d_exon_df[rows_to_update, "CI_high"] <- eff_size$CI_high
}

head(d_exon_df)


,.y.,group1,group2,n1,n2,statistic,p,p.adj,p.adj.signif,r_rank_biserial,CI,CI_low,CI_high
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,age_of_onset,8,14,34,47,0.9161197,0.359604137,1.0000000,ns,-0.1132666,0.95,-0.3540692,0.1416208
2,age_of_onset,8,15,34,6,-0.3594223,0.719279195,1.0000000,ns,0.1127451,0.95,-0.3748661,0.5514940
3,age_of_onset,8,37,34,9,-1.0690420,0.285050760,1.0000000,ns,0.2189542,0.95,-0.2036497,0.5727811
4,age_of_onset,8,43,34,8,0.7746816,0.438527800,1.0000000,ns,-0.1102941,0.95,-0.5084719,0.3267428
5,age_of_onset,8,44,34,13,-0.3995358,0.689498438,1.0000000,ns,0.1221719,0.95,-0.2450302,0.4587202
6,age_of_onset,8,45,34,6,-3.4779289,0.000505304,0.1167252,ns,0.8676471,0.95,0.6729864,0.9499024


In [47]:
head(domain_df)

,individual_id,age_of_onset,domain
,<int>,<dbl>,<fct>
1,4,2,TM
2,5,10,TM
3,6,2,TM
4,7,10,TM
5,19,12,BSol
6,28,6,TM


In [56]:
domain_factors <- levels(domain_df$domain)
c2_domain <- combn(domain_factors, 2, simplify = FALSE)
d_domain <- dunn_test(age_of_onset ~ domain, data = domain_df, p.adjust.method = "bonferroni")

# convert the dunn test results to a data frame
d_domain_df <- as.data.frame(d_domain)
# add columns, r_rank_biserial, CI, CI_low, CI_high
d_domain_df$r_rank_biserial <- NA
d_domain_df$CI <- NA
d_domain_df$CI_low <- NA
d_domain_df$CI_high <- NA


for (i in seq_along(c2_domain)) {
  domain1 <- c2_domain[[i]][1]
  domain2 <- c2_domain[[i]][2]

  # filter the data
  x_domain <- domain_df[domain_df$domain == domain1, "age_of_onset", drop = FALSE]
  y_domain <- domain_df[domain_df$domain == domain2, "age_of_onset", drop = FALSE]

  # calculate the effect size
  eff_size_domain <- rank_biserial(x = x_domain$age_of_onset, y = y_domain$age_of_onset)

  # Store the effect size with the dunn test results
  rows_to_update_domain <- d_domain_df$group1 == domain1 & d_domain_df$group2 == domain2 |
    d_domain_df$group1 == domain2 & d_domain_df$group2 == domain1

  d_domain_df[rows_to_update_domain, "r_rank_biserial"] <- eff_size_domain$r_rank_biserial
  d_domain_df[rows_to_update_domain, "CI"] <- eff_size_domain$CI
  d_domain_df[rows_to_update_domain, "CI_low"] <- eff_size_domain$CI_low
  d_domain_df[rows_to_update_domain, "CI_high"] <- eff_size_domain$CI_high
}

head(d_domain_df)

,.y.,group1,group2,n1,n2,statistic,p,p.adj,p.adj.signif,r_rank_biserial,CI,CI_low,CI_high
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,age_of_onset,BSol,CSol,92,75,-2.1872594,0.02872359,0.8042606,ns,0.18884058,0.95,0.01456331,0.35198255
2,age_of_onset,BSol,CTD,92,26,0.2149391,0.82981476,1.0000000,ns,-0.02842809,0.95,-0.27367358,0.22028823
3,age_of_onset,BSol,JSol,92,9,-0.9074742,0.36415609,1.0000000,ns,0.17270531,0.95,-0.21910185,0.51654810
4,age_of_onset,BSol,NTD,92,105,1.5878219,0.11232661,1.0000000,ns,-0.12101449,0.95,-0.27624903,0.04037372
5,age_of_onset,BSol,SPRY,92,10,0.1943119,0.84593169,1.0000000,ns,-0.05217391,0.95,-0.40603045,0.31528788
6,age_of_onset,BSol,TM,92,63,-0.2047308,0.83778244,1.0000000,ns,0.01552795,0.95,-0.16849452,0.19850447


In [57]:
# same for subdomain
subdomain_factors <- levels(subdomain_df$subdomain)
c2_subdomain <- combn(subdomain_factors, 2, simplify = FALSE)
d_subdomain <- dunn_test(age_of_onset ~ subdomain, data = subdomain_df, p.adjust.method = "bonferroni")

# convert the dunn test results to a data frame
d_subdomain_df <- as.data.frame(d_subdomain)
# add columns, r_rank_biserial, CI, CI_low, CI_high
d_subdomain_df$r_rank_biserial <- NA
d_subdomain_df$CI <- NA
d_subdomain_df$CI_low <- NA
d_subdomain_df$CI_high <- NA

for (i in seq_along(c2_subdomain)) {
  subdomain1 <- c2_subdomain[[i]][1]
  subdomain2 <- c2_subdomain[[i]][2]

  # filter the data
  x_subdomain <- subdomain_df[subdomain_df$subdomain == subdomain1, "age_of_onset", drop = FALSE]
  y_subdomain <- subdomain_df[subdomain_df$subdomain == subdomain2, "age_of_onset", drop = FALSE]

  # calculate the effect size
  eff_size_subdomain <- rank_biserial(x = x_subdomain$age_of_onset, y = y_subdomain$age_of_onset)

  # Store the effect size with the dunn test results
  rows_to_update_subdomain <- d_subdomain_df$group1 == subdomain1 & d_subdomain_df$group2 == subdomain2 |
    d_subdomain_df$group1 == subdomain2 & d_subdomain_df$group2 == subdomain1

  d_subdomain_df[rows_to_update_subdomain, "r_rank_biserial"] <- eff_size_subdomain$r_rank_biserial
  d_subdomain_df[rows_to_update_subdomain, "CI"] <- eff_size_subdomain$CI
  d_subdomain_df[rows_to_update_subdomain, "CI_low"] <- eff_size_subdomain$CI_low
  d_subdomain_df[rows_to_update_subdomain, "CI_high"] <- eff_size_subdomain$CI_high
}

head(d_subdomain_df)

,.y.,group1,group2,n1,n2,statistic,p,p.adj,p.adj.signif,r_rank_biserial,CI,CI_low,CI_high
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,age_of_onset,BSol1,CSol,90,70,-2.3540919,0.01856802,1,ns,0.20619048,0.95,0.02828535,0.37143425
2,age_of_onset,BSol1,CTD,90,23,0.1381121,0.89015185,1,ns,-0.02125604,0.95,-0.27920014,0.23954928
3,age_of_onset,BSol1,EF1&2,90,5,1.3043475,0.19211513,1,ns,-0.29777778,0.95,-0.68033447,0.21229905
4,age_of_onset,BSol1,JSol,90,9,-0.8032342,0.42183936,1,ns,0.15432099,0.95,-0.23741683,0.50288672
5,age_of_onset,BSol1,NSol,90,50,1.4581062,0.14481128,1,ns,-0.13022222,0.95,-0.31966185,0.06922772
6,age_of_onset,BSol1,NTD-A,90,46,1.1338250,0.25686796,1,ns,-0.11038647,0.95,-0.30650784,0.09472544


In [59]:
# print out only significant results
d_exon_sig <- d_exon_df[d_exon_df$p.adj < 0.05,]

d_exon_sig

,.y.,group1,group2,n1,n2,statistic,p,p.adj,p.adj.signif,r_rank_biserial,CI,CI_low,CI_high
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
26,age_of_onset,14,45,47,6,-4.028168,5.621306e-05,0.012985216,*,0.9716312,0.95,0.9254162,0.9893677
40,age_of_onset,14,102,47,6,-3.738223,1.853253e-04,0.042810146,*,0.9255319,0.95,0.8114013,0.9716763
117,age_of_onset,45,89,6,6,3.938957,8.183659e-05,0.018904251,*,-1.0000000,0.95,-1.0000000,-1.0000000
119,age_of_onset,45,93,6,8,4.319043,1.567075e-05,0.003619943,**,-1.0000000,0.95,-1.0000000,-1.0000000
120,age_of_onset,45,94,6,8,3.756981,1.719755e-04,0.039726329,*,-1.0000000,0.95,-1.0000000,-1.0000000
194,age_of_onset,89,102,6,6,-3.721241,1.982459e-04,0.045794810,*,1.0000000,0.95,1.0000000,1.0000000
209,age_of_onset,93,102,8,6,-4.086295,4.383169e-05,0.010125121,*,0.9791667,0.95,0.9280800,0.9940766


In [60]:
# print out only significant results
d_domain_sig <- d_domain_df[d_domain_df$p.adj < 0.05,]
d_domain_sig

,.y.,group1,group2,n1,n2,statistic,p,p.adj,p.adj.signif,r_rank_biserial,CI,CI_low,CI_high
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
10,age_of_onset,CSol,NTD,75,105,3.750535,0.0001764577,0.004940815,**,-0.3381587,0.95,-0.4804478,-0.1785241


In [61]:
# print out only significant results
d_subdomain_sig <- d_subdomain_df[d_subdomain_df$p.adj < 0.05,]
d_subdomain_sig

,.y.,group1,group2,n1,n2,statistic,p,p.adj,p.adj.signif,r_rank_biserial,CI,CI_low,CI_high
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
15,age_of_onset,CSol,NSol,70,50,3.415042,0.0006377229,0.04208971,*,-0.394,0.95,-0.5559322,-0.2032556
